In [1]:
import tushare as ts
from tqdm.notebook import tqdm
import sqlite3
import datetime
import pickle
import pandas as pd
from junnko_backtest import *

In [2]:
with open('cols_for_create_tables.pkl', 'rb') as f:
    cols = pickle.load(f)
    cols['index_daily_price'] = cols['stock_daily_price']

In [3]:
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# 建表

In [4]:
for table in ['stock_info', 'index_info']:
    sql_exp1 = f"""
    CREATE TABLE {table} (
    """
    sql_exp2 = """,
        INSERT_TIME DEFAULT CURRENT_TIMESTAMP,
        PRIMARY KEY (ts_code)
        );
    """
    sql_exp = sql_exp1+ ',\n'.join(cols[table]) + sql_exp2

    cursor.execute(sql_exp)

for table in ['stock_daily_price', 'stock_daily_basic', 'index_daily_price']:
    sql_exp1 = f"""
    CREATE TABLE {table} (
    """
    sql_exp2 = """,
        INSERT_TIME DEFAULT CURRENT_TIMESTAMP,
        PRIMARY KEY (ts_code, trade_date),
        FOREIGN KEY (ts_code) REFERENCES stock_info(ts_code)
        );
    """
    sql_exp = sql_exp1+ ',\n'.join(cols[table]) + sql_exp2

    cursor.execute(sql_exp)

for table in ['income', 'balancesheet', 'cashflow']:
    sql_exp1 = f"""
    CREATE TABLE {table} (
    """
    sql_exp2 = """,
        INSERT_TIME DEFAULT CURRENT_TIMESTAMP,
        PRIMARY KEY (ts_code, ann_date, f_ann_date, end_date, report_type, comp_type, end_type, update_flag),
        FOREIGN KEY (ts_code) REFERENCES stock_info(ts_code)
        );
    """
    sql_exp = sql_exp1+ ',\n'.join(cols[table]) + sql_exp2

    cursor.execute(sql_exp)

table = 'trade_cal'
if table:
    cursor.execute('''
    CREATE TABLE trade_cal (
        exchange,
        cal_date,
        is_open,
        pretrade_date,
        INSERT_TIME DEFAULT CURRENT_TIMESTAMP,
        PRIMARY KEY (exchange, cal_date)
    );
    ''')

table = 'index_component'
if table:
    sql_exp1 = f"""
    CREATE TABLE {table} (
    """
    sql_exp2 = """,
        INSERT_TIME DEFAULT CURRENT_TIMESTAMP,
        PRIMARY KEY (index_code, con_code, trade_date),
        FOREIGN KEY (index_code) REFERENCES index_info(ts_code),
        FOREIGN KEY (con_code) REFERENCES stock_info(ts_code)
        );
    """
    sql_exp = sql_exp1+ ',\n'.join(cols[table]) + sql_exp2

    cursor.execute(sql_exp)

# 获取数据

In [5]:
if 'stock_info':
    df1 = pro.stock_basic(exchange='', list_status='L', fields=cols['stock_info'])
    df2 = pro.stock_basic(exchange='', list_status='D', fields=cols['stock_info'])
    df3 = pro.stock_basic(exchange='', list_status='P', fields=cols['stock_info'])
    df = pd.concat([df1, df2, df3], axis=0)
    df.to_sql('stock_info', conn, if_exists='append', index=False)
    all_codes = df['ts_code'].unique().tolist()

In [6]:
query = f'select distinct ts_code from stock_info'
all_codes = pd.read_sql_query(query, conn)['ts_code'].tolist()
len(all_codes)

5609

In [7]:
if 'index_info':
    dfs = []
    for market in ['CSI', 'SSE', 'SZSE', 'CICC', 'SW', 'OTH']:
        df = pro.index_basic(market=market, fields=cols['index_info'])
        dfs.append(df)
    df = pd.concat(dfs, axis=0)
    df.to_sql('index_info', conn, if_exists='append', index=False)

In [4]:
if 'trade_cal':
    df1 = pro.trade_cal(exchange='SSE')
    df2 = pro.trade_cal(exchange='SZSE')
    df = pd.concat([df1, df2], axis=0)
    df.to_sql('trade_cal', conn, if_exists='append', index=False)

In [ ]:
if 'stock_daily_price':
    dfs = []
    for code in tqdm(all_codes):
        df = ts.pro_bar(ts_code=code, adj='qfq', start_date='20000101')
        if isinstance(df, pd.DataFrame):
            dfs.append(df)
    df = pd.concat(dfs, axis=0)
    df.to_sql('stock_daily_price', conn, if_exists='append', index=False)

In [ ]:
if 'stock_daily_basic':
    dfs = []
    for code in tqdm(all_codes):
        df = pro.daily_basic(ts_code=code, fields=cols['stock_daily_basic'], start_date='20000101')
        if isinstance(df, pd.DataFrame):
            dfs.append(df)
    df = pd.concat(dfs, axis=0)
    df.to_sql('stock_daily_basic', conn, if_exists='append', index=False)

In [ ]:
if 'income':
    dfs = []
    for code in tqdm(all_codes):
        time1 = time.time()
        df = pro.income(ts_code=code, fields=cols['income'])
        if isinstance(df, pd.DataFrame):
            dfs.append(df)
        time2 = time.time()
        iter_rate_limiter(max_iter_per_min=500, actual_iter_time=time2-time1)
    df = pd.concat(dfs, axis=0)
    df.to_sql('income', conn, if_exists='append', index=False)

In [ ]:
if 'balancesheet':
    dfs = []
    for code in tqdm(all_codes):
        time1 = time.time()
        df = pro.balancesheet(ts_code=code, fields=cols['balancesheet'])
        if isinstance(df, pd.DataFrame):
            dfs.append(df)
        time2 = time.time()
        iter_rate_limiter(max_iter_per_min=500, actual_iter_time=time2-time1)
    df = pd.concat(dfs, axis=0)
    df.to_sql('balancesheet', conn, if_exists='append', index=False)

In [ ]:
if 'cashflow':
    dfs = []
    for code in tqdm(all_codes):
        time1 = time.time()
        df = pro.cashflow(ts_code=code, fields=cols['cashflow'])
        if isinstance(df, pd.DataFrame):
            dfs.append(df)
        time2 = time.time()
        iter_rate_limiter(max_iter_per_min=500, actual_iter_time=time2-time1)
    df = pd.concat(dfs, axis=0)
    df.to_sql('cashflow', conn, if_exists='append', index=False)